## 과제 1
ReLu activation function과 derivative function을 구현해보세요
- Hint : np.maximum 함수 사용하면 편리합니다
- 다른 방법 사용하셔도 무방합니다


In [ ]:
import numpy as np 

In [ ]:
import numpy as np 
def relu(x):
  
  x= np.maximum (0,x)

  return x

In [37]:
def d_relu(x):
  x= np.maximum (0,x)
  if x >0 :
    x=1 
    return x
  else :
    x=0
    return x


In [44]:
x0= torch.tensor([1,2,3,4,-1,-2,-3])
d_relu(x0)

RuntimeError: ignored

## 과제 2
Deep Learning Basic 코드 파일의 MLP implementation with Numpy library using MNIST dataset 코드 참고해서
Three layer MLP 일 때의 backward_pass 함수를 완성해주세요.   
- Hint : 코드 파일의 예시는 Two layer MLP


In [ ]:
def foward_pass(x, params):
  
  params["S1"] = np.dot(params["W1"], x) + params["b1"]
  params["A1"] = sigmoid(params["S1"])
  params["S2"] = np.dot(params["W2"], params["A1"]) + params["b2"]
  params["A2"] = sigmoid(params["S2"])
  params["S3"] = np.dot(params["W3"], params["A2"]) + params["b3"]
  params["A3"] = softmax(params[S3])

  return params

In [ ]:
def backward_pass(x, y_true, params):

  dS3 = params["A3"] - y_true
  grads = {}

  grads["dW3"] =  np.dot(dS3, params["A2"].T)/x.shape[1]
  grads["db3"] =  (1/x.shape[1])*np.sum(dS3, axis=1, keepdims=True)/x.shape[1]

  dA2 = np.dot(params["W3"].T, dS3)
  dS2 = dA2 * d_sigmoid(params["S2"])

  grads["dW2"] = np.dot(dS2, params["A1"].T)/x.shape[1]
  grads["db2"] = (1/x.shape[1])*np.sum(dS2, axis=1, keepdims=True)/x.shape[1]

  dA1= np.dot(params["W2"].T , dS2)
  dS1= dA1 * d_sigmoid(params["S1"])


  grads["dW1"] = np.dot(dS1, x.T)/x.shape[1]
  grads["db1"] = np.sum(dS1, axis=1, keepdims=True)/x.shape[1]

  return grads

## 과제 3
Deep Learning Basic 코드 파일의 MLP implementation with Pytorch library using MNIST dataset 코드 참고해서
Three layer MLP를 구한후, 학습을 돌려 보세요

hyperparameter는 다음과 같이 설정

- epochs : 100
- hiddensize : 128, 64 (two layer)
- learning_rate : 0.5

In [ ]:
# Assignment 3 구현은 여기서 ()

In [ ]:
from torchvision import transforms, datasets
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [ ]:
# 이미지를 텐서로 변경
transform = transforms.Compose([
    transforms.ToTensor()
])

In [ ]:
trainset = datasets.MNIST(
    root      = './.data/', 
    train     = True,
    download  = True,
    transform = transform
)
testset = datasets.MNIST(
    root      = './.data/', 
    train     = False,
    download  = True,
    transform = transform
)

In [ ]:
BATCH_SIZE = 512
# train set과 test set 각각에 대하여 DataLoader를 생성합니다.
# shuffle=True 매개변수를 넣어 데이터를 섞어주세요.
train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
test_loader =  DataLoader(testset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = nn.Linear(784,128)
        self.layer2 = nn.Linear(128,64)
        self.layer3 = nn.Linear(64,10)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = x.view(-1, 784)
        out = self.layer1(x)
        out = self.relu(out)
        out = self.layer2(out)
        out = self.relu(out)
        out = self.layer3(out)

        return out

In [ ]:
model = Net()
model

Net(
  (layer1): Linear(in_features=784, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=64, bias=True)
  (layer3): Linear(in_features=64, out_features=10, bias=True)
  (relu): ReLU()
)

In [ ]:
list(model.parameters()) # 행렬들을 직접 살펴볼 수 있음
                         # require_true 얘는 학습되는 애구나 알 수 있음

[Parameter containing:
 tensor([[ 0.0110, -0.0344,  0.0251,  ..., -0.0067,  0.0095,  0.0262],
         [-0.0177,  0.0293,  0.0322,  ...,  0.0029, -0.0178,  0.0119],
         [ 0.0227,  0.0264, -0.0101,  ...,  0.0287,  0.0271,  0.0071],
         ...,
         [-0.0150, -0.0054,  0.0009,  ...,  0.0278,  0.0046,  0.0333],
         [-0.0304,  0.0125, -0.0286,  ..., -0.0022,  0.0214,  0.0153],
         [ 0.0077,  0.0133, -0.0136,  ...,  0.0145, -0.0044, -0.0185]],
        requires_grad=True), Parameter containing:
 tensor([-2.4017e-02,  3.8905e-03, -1.7750e-02,  8.5210e-03,  1.1645e-02,
          6.1743e-03,  9.5233e-03,  3.3894e-02,  2.8637e-02, -2.8677e-02,
         -2.5669e-02,  1.4855e-02, -1.0258e-02, -1.3765e-02,  1.7460e-02,
         -1.9524e-02, -2.9866e-02, -2.7430e-02, -2.7848e-02, -5.1663e-03,
         -1.5489e-02, -1.8940e-02,  7.1803e-03,  4.2274e-03, -8.0835e-03,
         -3.0267e-02, -2.5036e-02, -1.1842e-02, -3.0982e-02,  1.6853e-02,
          6.6013e-03, -1.5906e-02, -2.354

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.5)

In [ ]:
def train(model, train_loader, optimizer):
    model.train()
    # 배치 당 loss 값을 담을 리스트 생성
    batch_losses = []

    for data, target in train_loader:
        # 옵티마이저의 기울기 초기화
        optimizer.zero_grad()

        # y pred 값 산출
        output = model(data)
        # loss 계산
        # 정답 데이터와의 cross entropy loss 계산
        # 이 loss를 배치 당 loss로 보관
        loss = criterion(output, target)
        batch_losses.append(loss)

        # 기울기 계산
        loss.backward()

        # 가중치 업데이트!
        optimizer.step()
        
    # 배치당 평균 loss 계산
    avg_loss = sum(batch_losses) / len(batch_losses)
    
    return avg_loss

In [ ]:
def evaluate(model, test_loader):
    # 모델을 평가 모드로 전환
    model.eval()

    batch_losses = []
    correct = 0 

    with torch.no_grad(): 
        for data, target in test_loader:
            # 예측값 생성
            output = model(data)

            # loss 계산 (이전과 동일)
            loss = criterion(output, target)
            batch_losses.append(loss)

           # Accuracy 계산
           # y pred와 y가 일치하면 correct에 1을 더해주기
            pred = output.max(1, keepdim=True)[1]

            # eq() 함수는 값이 일치하면 1을, 아니면 0을 출력.
            correct += pred.eq(target.view_as(pred)).sum().item()

    # 배치 당 평균 loss 계산 
    avg_loss =  sum(batch_losses) / len(batch_losses)

    #정확도 계산
    accuracy = 100. * correct / len(test_loader.dataset)

    return avg_loss, accuracy

In [ ]:
EPOCHS = 100

for epoch in range(1, EPOCHS + 1):
    train_loss = train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Train Loss: {:.4f}\tTest Loss: {:.4f}\tAccuracy: {:.2f}%'.format(
          epoch, train_loss, test_loss, test_accuracy))

[1] Train Loss: 0.8365	Test Loss: 0.2869	Accuracy: 91.30%
[2] Train Loss: 0.2499	Test Loss: 0.2225	Accuracy: 93.33%
[3] Train Loss: 0.2358	Test Loss: 0.1817	Accuracy: 94.13%
[4] Train Loss: 0.1408	Test Loss: 0.1862	Accuracy: 93.98%
[5] Train Loss: 0.1177	Test Loss: 0.1488	Accuracy: 95.18%
[6] Train Loss: 0.0953	Test Loss: 0.1017	Accuracy: 96.89%
[7] Train Loss: 0.0815	Test Loss: 0.0895	Accuracy: 97.03%
[8] Train Loss: 0.0715	Test Loss: 0.0939	Accuracy: 97.05%
[9] Train Loss: 0.0616	Test Loss: 0.0856	Accuracy: 97.44%


KeyboardInterrupt: ignored

## 과제 4
과제 3 부분의 성능을 지금까지 배운 지식을 바탕으로 향상시켜보세요

- Hint : Activation function, hyperparameter setting

In [ ]:
# Assignment 4 구현은 여기서 ()

In [ ]:
BATCH_SIZE = 64
# train set과 test set 각각에 대하여 DataLoader를 생성합니다.
# shuffle=True 매개변수를 넣어 데이터를 섞어주세요.
train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
test_loader =  DataLoader(testset, batch_size=BATCH_SIZE, shuffle=True)

In [46]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

In [ ]:
class Net_2(nn.Module):
    def __init__(self):
        super(Net_2, self).__init__()
        self.layer1 = nn.Linear(784,128)
        torch.nn.init.xavier_uniform_(self.layer1.weight)
        self.layer2 = nn.Linear(128,64)
        torch.nn.init.xavier_uniform_(self.layer2.weight)
        self.layer3 = nn.Linear(64,10)
        self.LeakyReLU = nn.LeakyReLU()
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = x.view(-1, 784)
        out = self.layer1(x)
        out = self.LeakyReLU(out)
        out = self.layer2(out)
        out = self.relu(out)
        out = self.layer3(out)

        return out

In [ ]:
model_2 = Net_2()

In [45]:
EPOCHS = 100

for epoch in range(1, EPOCHS + 1):
    train_loss = train(model_2, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Train Loss: {:.4f}\tTest Loss: {:.4f}\tAccuracy: {:.2f}%'.format(
          epoch, train_loss, test_loss, test_accuracy))

[1] Train Loss: 2.3144	Test Loss: 0.0793	Accuracy: 97.52%
[2] Train Loss: 2.3143	Test Loss: 0.0779	Accuracy: 97.52%
[3] Train Loss: 2.3144	Test Loss: 0.0783	Accuracy: 97.52%
[4] Train Loss: 2.3144	Test Loss: 0.0783	Accuracy: 97.52%
[5] Train Loss: 2.3143	Test Loss: 0.0779	Accuracy: 97.52%
[6] Train Loss: 2.3143	Test Loss: 0.0779	Accuracy: 97.52%
[7] Train Loss: 2.3143	Test Loss: 0.0796	Accuracy: 97.52%
[8] Train Loss: 2.3144	Test Loss: 0.0779	Accuracy: 97.52%
[9] Train Loss: 2.3143	Test Loss: 0.0779	Accuracy: 97.52%
[10] Train Loss: 2.3144	Test Loss: 0.0780	Accuracy: 97.52%
[11] Train Loss: 2.3144	Test Loss: 0.0779	Accuracy: 97.52%
[12] Train Loss: 2.3144	Test Loss: 0.0780	Accuracy: 97.52%
[13] Train Loss: 2.3144	Test Loss: 0.0780	Accuracy: 97.52%
[14] Train Loss: 2.3144	Test Loss: 0.0779	Accuracy: 97.52%
[15] Train Loss: 2.3143	Test Loss: 0.0780	Accuracy: 97.52%
[16] Train Loss: 2.3143	Test Loss: 0.0779	Accuracy: 97.52%
[17] Train Loss: 2.3144	Test Loss: 0.0779	Accuracy: 97.52%
[18] T

In [ ]:
#loss 가 잘 줄어들지 않아 Lr을 조정하였습니다. 
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

In [47]:
EPOCHS = 100

for epoch in range(1, EPOCHS + 1):
    train_loss = train(model_2, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Train Loss: {:.4f}\tTest Loss: {:.4f}\tAccuracy: {:.2f}%'.format(
          epoch, train_loss, test_loss, test_accuracy))

[1] Train Loss: 2.3143	Test Loss: 0.0791	Accuracy: 97.52%
[2] Train Loss: 2.3143	Test Loss: 0.0783	Accuracy: 97.52%
[3] Train Loss: 2.3144	Test Loss: 0.0784	Accuracy: 97.52%
[4] Train Loss: 2.3144	Test Loss: 0.0789	Accuracy: 97.52%


KeyboardInterrupt: ignored

batch: 60000, lr=0.01, xavier initialization for layer1, 2 results in about 94...and more





**무엇을 보완하였고, 왜 보완되었는지에 대한 자유 서술 (아래에)**

1. optimizer - > SGD 에서 adam으로 바꿔보도록 하였습니다
-> 보편화된 optimizer는 adam을 주로 설정하기 때문입니다
 2. learning rate를 낮추도록 조정하였습니다.  
 -> 이유는 accuracy가 수렴하지 못하고 학습동안 느리게 수렴하였기 때문입니다. 

 3.  활성화 함수로 Leaky ReLu 를 추가하였습니다! 
 ->  

4. 가중치 초기화를 진행하였습니다 
->  초기값 설정이 학습과정에 매우 큰 영향을 끼칠 수 있기 때문에

